In [151]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import os

# 1. Codeup Blog Articles

Visit Codeup's Blog and record the urls for at least 5 distinct blog posts. For each post, you should scrape at least the post's title and content.

Encapsulate your work in a function named get_blog_articles that will return a list of dictionaries, with each dictionary representing one article. The shape of each dictionary should look like this:


{
    'title': 'the title of the article',
    'content': 'the full text content of the article'
}
Plus any additional properties you think might be helpful.

**Bonus**: Scrape the text of all the articles linked on codeup's blog page.

In [2]:
url = 'https://codeup.com/blog/'
headers = {'User-Agent': 'Codeup Data Science'}
response = get(url, headers=headers)

In [11]:
soup = BeautifulSoup(response.content, 'html.parser')

In [59]:
def get_blog_links(soup, existing_list):
    blog_list = existing_list
    for a in soup.select('a'):
        if 'read more' in a.text:
            blog_list.append(a['href'])
    return blog_list

In [51]:
def get_next_page(soup):
    for a in soup.select('a'):
        if 'Older Entries' in a.text:
            return a['href']

In [52]:
def get_soup(url):
    headers = {'User-Agent': 'Codeup Data Science'}
    response =  get(url, headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [54]:
soup = get_soup(url)
links = get_blog_links(soup)
links

['https://codeup.com/workshops/in-person-workshop-learn-to-code-python-on-7-19/',
 'https://codeup.com/workshops/dallas/free-javascript-workshop-at-codeup-dallas-on-6-28/',
 'https://codeup.com/tips-for-prospective-students/is-our-cloud-administration-program-right-for-you/',
 'https://codeup.com/workshops/pride-in-tech-panel/',
 'https://codeup.com/codeup-news/inclusion-at-codeup-during-pride-month-and-always/',
 'https://codeup.com/tips-for-prospective-students/mental-health-first-aid-training/',
 'https://codeup.com/workshops/codeup-dallas-how-to-succeed-at-a-coding-bootcamp-on-june-9th/',
 'https://codeup.com/featured/5-reasons-to-attend-our-new-cloud-administration-program/',
 'https://codeup.com/workshops/learn-to-code-python-on-5-21/',
 'https://codeup.com/dallas-newsletter/codeup-dallas-joins-career-day/',
 'https://codeup.com/workshops/project-quest-info-session-may-24th/',
 'https://codeup.com/workshops/from-bootcamp-to-bootcamp-a-military-appreciation-panel/',
 'https://code

In [55]:
url = get_next_page(soup)
url

'https://codeup.com/blog/page/2/?et_blog'

In [65]:
def get_all_blog_links(initial_url):
    existing_list = []
    url = initial_url
    while True:
        soup = get_soup(url)
        existing_list = get_blog_links(soup, existing_list)
        url = get_next_page(soup)
        if url == None:
            return existing_list

In [66]:
master_list = get_all_blog_links('https://codeup.com/blog/')

In [68]:
test = get_soup(master_list[0])

In [73]:
test.title.text

'In-Person Workshop: Learn to Code - Python on 7/19 - Codeup'

In [154]:
def get_all_info():
    master_list = get_all_blog_links('https://codeup.com/blog/')
    title = []
    content = []
    link = []
    date = []
    for i in range(len(master_list)):
        page = get_soup(master_list[i])
        date.append(page('p')[0].text.split('|')[0])
        title.append(page.title.text)
        link.append(master_list[i])
        content.append(page.article.text.strip())
    all_info = {'title':title, 'date':date, 'content':content, 'link':link}
    df = pd.DataFrame(all_info)
    return df

In [155]:
df = get_all_info()

In [163]:
df.to_csv('master_blog_list.csv', index=False)

In [167]:
df.head()

,title,date,content,link
0,In-Person Workshop: Learn to Code - Python on ...,"Jun 20, 2022",In-Person Workshop: Learn to Code – Python on ...,https://codeup.com/workshops/in-person-worksho...
1,Free JavaScript Workshop at Codeup Dallas on 6...,"Jun 19, 2022",Free JavaScript Workshop at Codeup Dallas on 6...,https://codeup.com/workshops/dallas/free-javas...
2,Is Our Cloud Administration Program Right for ...,"Jun 8, 2022",Is Our Cloud Administration Program Right for ...,https://codeup.com/tips-for-prospective-studen...
3,PRIDE in Tech Panel - Codeup,"Jun 5, 2022","PRIDE in Tech Panel\nJun 5, 2022 | Dallas, San...",https://codeup.com/workshops/pride-in-tech-panel/
4,Inclusion at Codeup During Pride Month (and Al...,"Jun 1, 2022",Inclusion at Codeup During Pride Month (and Al...,https://codeup.com/codeup-news/inclusion-at-co...


In [229]:
os.path.isfile('master_blog_list.csv') == False

False

# 2. News Articles

We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.

Write a function that scrapes the news articles for the following topics:

Business
Sports
Technology
Entertainment
The end product of this should be a function named get_news_articles that returns a list of dictionaries, where each dictionary has this shape:


{
    'title': 'The article title',
    'content': 'The article content',
    'category': 'business' # for example
}
Hints:

Start by inspecting the website in your browser. Figure out which elements will be useful.
Start by creating a function that handles a single article and produces a dictionary like the one above.
Next create a function that will find all the articles on a single page and call the function you created in the last step for every article on the page.
Now create a function that will use the previous two functions to scrape the articles from all the pages that you need, and do any additional processing that needs to be done.


In [172]:
base_url = 'https://inshorts.com/en/read/'
category_list = ['business', 'sports', 'technology', 'entertainment']

In [173]:
url = base_url + category_list[0]
headers = {'User-Agent': 'Codeup Data Science'}
response = get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [211]:
business = soup.select('div[class = "news-card z-depth-1"]')
for i in range(len(business)):
    print(business[i].select('span[itemprop = "headline"]')[0].text)
    print(business[i].select('div[itemprop="articleBody"]')[0].text)

Rupee edges closer to 80 per US dollar, opens at record low of 79.94
The rupee on Friday opened at a record low of 79.94 against the US dollar. So far this year, the US dollar is up 7% against the Indian currency. On Thursday, the currency had closed at another record low of 79.90 against the US dollar. Meanwhile, BSE Sensex jumped 316 points to 53,733 in Friday's early session. 
Rupee drops 9 paise to close at all-time low of 79.90 against US dollar
The rupee declined by 9 paise to close at a new record low of 79.90 against the US dollar on Thursday. At the interbank foreign exchange market, the local currency opened at 79.72 and witnessed intra-day high of 79.71 and a low of 79.92 against the US dollar. Meanwhile, brent crude futures, the global oil benchmark, dropped 2.20% to $97.38/barrel.
Like all firms, we are not immune to economic headwinds: Google CEO to staff
In an internal memo to the staff, Google CEO Sundar Pichai while announcing the company’s plan to slow its hiring, sai

In [220]:
def get_all_articles():
    # Setting base variables
    base_url = 'https://inshorts.com/en/read/'
    category_list = ['business', 'sports', 'technology', 'entertainment']
    full_list = []
    # Making a for loop to go to the pages of all the categories
    for cat in category_list:
        # Getting the soup object
        url = base_url + cat
        headers = {'User-Agent': 'Codeup Data Science'}
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Getting the page articles
        page = soup.select('div[class = "news-card z-depth-1"]')
        # Getting the article title and content for the page
        for i in range(len(page)):
            single = {'title': (page[i].select('span[itemprop = "headline"]')[0].text), 
                      'content': (page[i].select('div[itemprop="articleBody"]')[0].text) , 
                      'category': cat}
            full_list.append(single)
    return full_list

In [221]:
final = get_all_articles()

[{'title': 'Rupee edges closer to 80 per US dollar, opens at record low of 79.94', 'content': "The rupee on Friday opened at a record low of 79.94 against the US dollar. So far this year, the US dollar is up 7% against the Indian currency. On Thursday, the currency had closed at another record low of 79.90 against the US dollar. Meanwhile, BSE Sensex jumped 316 points to 53,733 in Friday's early session. ", 'category': 'business'}]
[{'title': 'Rupee edges closer to 80 per US dollar, opens at record low of 79.94', 'content': "The rupee on Friday opened at a record low of 79.94 against the US dollar. So far this year, the US dollar is up 7% against the Indian currency. On Thursday, the currency had closed at another record low of 79.90 against the US dollar. Meanwhile, BSE Sensex jumped 316 points to 53,733 in Friday's early session. ", 'category': 'business'}, {'title': 'Rupee drops 9 paise to close at\xa0all-time low of 79.90 against US dollar', 'content': 'The rupee declined by 9 pais

[{'title': 'Rupee edges closer to 80 per US dollar, opens at record low of 79.94', 'content': "The rupee on Friday opened at a record low of 79.94 against the US dollar. So far this year, the US dollar is up 7% against the Indian currency. On Thursday, the currency had closed at another record low of 79.90 against the US dollar. Meanwhile, BSE Sensex jumped 316 points to 53,733 in Friday's early session. ", 'category': 'business'}, {'title': 'Rupee drops 9 paise to close at\xa0all-time low of 79.90 against US dollar', 'content': 'The rupee declined by 9 paise to close at a new record low of 79.90 against the US dollar on Thursday. At the interbank foreign exchange market, the local currency opened at 79.72 and witnessed intra-day high of 79.71 and a low of 79.92 against the US dollar. Meanwhile, brent crude futures, the global oil benchmark, dropped 2.20% to $97.38/barrel.', 'category': 'business'}, {'title': "Musk accused lawyers of causing trouble by seeking deal's financing info: Tw

In [225]:
df = pd.DataFrame(final)

In [226]:
df.to_csv('in_shorts_articles.csv', index=False)

In [227]:
df

,title,content,category
0,"Rupee edges closer to 80 per US dollar, opens ...",The rupee on Friday opened at a record low of ...,business
1,Rupee drops 9 paise to close at all-time low o...,The rupee declined by 9 paise to close at a ne...,business
2,Musk accused lawyers of causing trouble by see...,"In its lawsuit against Tesla CEO Elon Musk, Tw...",business
3,No plans for company-wide layoffs: Twitter ami...,"In a filing, Twitter has said that it isn't lo...",business
4,Elon has a real record of success: LinkedIn Co...,LinkedIn's billionaire Co-founder Reid Hoffman...,business
...,...,...,...
95,My dad likes my films when I'm not remaking hi...,Actress Sara Ali Khan recently appeared on cel...,entertainment
96,Bagwati in background: Abhay on 11 yrs of 'Zin...,Actor Abhay Deol shared a series of selfies on...,entertainment
97,"'Chup' is a commercial thriller, will release ...",Director R Balki has said his upcoming film 'C...,entertainment
98,"Playing quintessential B'wood hero is hardest,...",Actor Ranbir Kapoor said playing a quintessent...,entertainment
